In [ ]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
QiskitRuntimeService.save_account(channel="ibm_cloud", 
                                  instance="ibm_brisbane",
                                  token="YOUR_TOKEN", 
                                  set_as_default=True,
                                  overwrite=True)

service = QiskitRuntimeService()

In [ ]:
# Select backend
backend = service.backend("ibm_brisbane") 
print(f"Using backend: {backend.name}")

In [ ]:
properties = backend.properties(refresh=True)  #
target = backend.target  
config = backend.configuration()

In [ ]:
# Print general backend information
print(f"Backend Name: {properties.backend_name}")
print(f"Last Update: {properties.last_update_date}")
print(f"Number of Qubits: {config.n_qubits}")
print(f"Basis Gates: {config.basis_gates}")

In [ ]:
print("\nSingle-Qubit Errors and Gate Times:")
single_qubit_gates = ['id', 'rz', 'sx', 'x']  
for gate in single_qubit_gates:
        print(f"\nGate: {gate}")
        for qubit in range(min(5, config.n_qubits)):  # Limit to first 5 qubits
                error = properties.gate_error(gate, [qubit]) if properties.gate_error(gate, [qubit]) is not None else "N/A"
                duration = target[gate][(qubit,)].duration * 1e9  # Convert to ns
                print(f"  Qubit {qubit}: Error = {error:.4f}, Duration = {duration:.1f} ns")

In [ ]:
for qpair in config.coupling_map[:5]:  # Limit to first 5 pairs for brevity
        print("\nTwo-Qubit Gate Errors and Times (CNOT):")
        error = properties.gate_error('ecr', qpair) if properties.gate_error('ecr', qpair) is not None else "N/A"
        duration = target['ecr'][tuple(qpair)].duration * 1e9 if (tuple(qpair) in target['ecr']) else "N/A"
        print(f"    Qubits {qpair}: Error = {error:.4f}, Duration = {duration:.1f} ns")

In [ ]:
print("\nReadout Errors:")
for qubit in range(min(5, config.n_qubits)):  # Limit to first 5 qubits
    error = properties.readout_error(qubit)
    print(f"  Qubit {qubit}: Readout Error = {error:.4f}")

In [ ]:
print("\nT1 and T2 Times (for ThermalRelaxationError):")
for qubit in range(min(5, config.n_qubits)):
    t1 = properties.qubit_property(qubit)['T1'][0] * 1e6  # Convert to microseconds
    t2 = properties.qubit_property(qubit)['T2'][0] * 1e6  # Convert to microseconds
    print(f"  Qubit {qubit}: T1 = {t1:.2f} µs, T2 = {t2:.2f} µs")

In [ ]:
# Create Bell State
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

In [ ]:
# Transpile
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

In [ ]:
# Run on real hardware
sampler = Sampler(mode=backend)
job = sampler.run([isa_circuit], shots=1000)
result = job.result()
counts = result[0].data.meas.get_counts()

In [ ]:
print(f"Results: {counts}")

# Calculate Bell state fidelity
total = sum(counts.values())
bell_fidelity = (counts.get('00', 0) + counts.get('11', 0)) / total
print(f"Bell state fidelity: {bell_fidelity:.3f}")